- Emma Rey Sánchez - 202110801
- Catalina Royo-Villanova Seguí - 202104665
# Proyecto Final

In [1]:
import cv2
import numpy as np

## Calibración de la cámara

In [2]:
# Tamaño del tablero circular (filas, columnas)
circle_board_size = (5, 4)

# Tamaño del círculo en el tablero (en milímetros)
circle_size_mm = 30

# Crear un objeto para almacenar las coordenadas 3D del tablero circular
objp = np.zeros((circle_board_size[0] * circle_board_size[1], 3), np.float32)

# Llenar las coordenadas 3D con los centros de los círculos
objp[:, :2] = np.mgrid[0:circle_board_size[0], 0:circle_board_size[1]].T.reshape(-1, 2) * circle_size_mm

# Listas para almacenar coordenadas 3D e 2D del tablero
obj_points = []  # Coordenadas 3D del mundo real
img_points = []  # Coordenadas 2D en la imagen

# Inicializar la captura de video desde la cámara de la Raspberry Pi
cap = cv2.VideoCapture(0)

while True:
    # Capturar un solo fotograma
    ret, frame = cap.read()
    if not ret:
        break

    # Convertir la imagen a escala de grises
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Intentar encontrar el patrón de círculos en la imagen
    ret, corners = cv2.findCirclesGrid(gray, circle_board_size, flags=cv2.CALIB_CB_ASYMMETRIC_GRID)

    # Si se encuentra el patrón, agregar las coordenadas 3D y 2D a las listas
    if ret:
        obj_points.append(objp)
        img_points.append(corners)

        # Dibujar los círculos en la imagen
        cv2.drawChessboardCorners(frame, circle_board_size, corners, ret)

    # Mostrar la imagen
    cv2.imshow('Calibración Circular', frame)

    # Esperar a que se presione 'Esc' para salir o 'c' para realizar la calibración
    key = cv2.waitKey(1)
    if key == 27:
        break
    elif key == ord('c') and len(obj_points) > 5:
        # Calibrar la cámara
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, gray.shape[::-1], None, None)

        # Guardar los parámetros de calibración en un archivo
        np.savez('calibration_params.npz', mtx=mtx, dist=dist)

        print('Calibración completada. Parámetros guardados en calibration_params.npz')
        break

# Liberar la captura de video y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()